# モデルパラメータ

## レッスンの目標
* `max_tokens` パラメータの役割を理解する
* `temperature` パラメータでモデル応答をコントロールする
* `stop_sequence` の目的を説明できるようにする


いつも通り、まずは `anthropic` SDK を import して API キーを読み込みます：


In [ ]:
from dotenv import load_dotenv
from anthropic import Anthropic

# 環境変数を読み込む
load_dotenv()

# "ANTHROPIC_API_KEY" 環境変数を自動的に探して使う
client = Anthropic()


## Max tokens

Claude にリクエストを送るとき、毎回必ず指定しなければならない必須パラメータが 3 つあります：

* `model`
* `max_tokens`
* `messages`

これまで、すべてのリクエストで `max_tokens` を指定してきましたが、何のためのものかはまだ説明していませんでした。

最初に送ったリクエストは次のようなものでした：

```python
our_first_message = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=500,
    messages=[
        {"role": "user", "content": "Hi there! Please write me a haiku about a pet chicken"}
    ]
)
```

では、`max_tokens` の目的は何でしょうか？


### トークン（Tokens）

ひとことで言うと、`max_tokens` は **Claude が応答で生成してよいトークン数の上限**を指定します。先に進む前に、トークンについて少し整理しましょう。

多くの大規模言語モデルは、単語をそのまま扱うのではなく、トークンと呼ばれる「単語のかけら（word-fragments）」の列として扱います。トークンは、Claude がテキストを処理し、理解し、生成するための小さな構成要素です。こちらがプロンプトを渡すと、まずトークン列に変換されてモデルに渡され、モデルは **1トークンずつ** 出力を生成していきます。

Claude の場合、1トークンは英語ではおおむね 3.5 文字程度に相当しますが、言語によって変動します。


### `max_tokens` を使う

`max_tokens` は、Claude が生成するトークン数の上限を設定できます。たとえば、詩を書かせて `max_tokens` を 10 にすると、Claude はトークン生成を始めますが **10 トークンに達した時点で即停止**します。結果として、途中で切れた（不完全な）出力になりがちです。試してみましょう：


In [ ]:
truncated_response = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=10,
    messages=[
        {"role": "user", "content": "Write me a poem"}
    ]
)
print(truncated_response.content[0].text)


Claude から返ってきたのは例えば次のようなものです：

> Here is a poem for you:
>
> The

上のコードを実行すると、同じように途中で切れた別の結果になる可能性が高いです。Claude は詩を書き始めましたが、10 トークン生成した時点で停止しました。


また、レスポンスの `stop_reason` を見ると、モデルが **なぜ** 生成を止めたのかを確認できます。このケースでは値が `"max_tokens"` になっており、最大トークン数の上限に達したため停止したことが分かります。


In [ ]:
truncated_response.stop_reason


もちろん、`max_tokens` をもっと大きくしてもう一度詩を書かせれば、今度は詩全体が返ってくる可能性が高いです：


In [ ]:
longer_poem_response = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=500,
    messages=[
        {"role": "user", "content": "Write me a poem"}
    ]
)
print(longer_poem_response.content[0].text)


`max_tokens=500` で Claude が生成した例は次の通りです：

```
Here is a poem for you:

Whispers of the Wind

The wind whispers softly,
Caressing my face with care.
Its gentle touch, a fleeting breath,
Carries thoughts beyond compare.

Rustling leaves dance in rhythm,
Swaying to the breeze's song.
Enchanting melodies of nature,
Peaceful moments linger long.

The wind's embrace, a soothing balm,
Calms the restless soul within.
Embracing life's fleeting moments,
As the wind's sweet song begins.
```



このレスポンスの `stop_reason` を見ると `"end_turn"` になっているはずです。これは、モデルが自然に書き終えた（これ以上続けることがない）ため停止した、という意味です。


In [ ]:
longer_poem_response.stop_reason


また、`max_tokens` を増やしたからといって、必ずそのトークン数まで生成されるわけでもありません。たとえばジョークを頼んで `max_tokens=1000` にしても、ほぼ確実に 1000 トークンよりずっと短い応答になります。

重要な点として、モデルは生成中に `max_tokens` を "理解" しているわけではありません。`max_tokens` を変えても生成の仕方そのものは変わらず、単に **生成できる余裕を与える（上限を大きくする）**か、**途中で切る（上限を小さくする）**かが変わるだけです。


上の例では、Claude に "Tell me a joke" と頼み、`max_tokens` に 1000 を指定しています。しかし、生成される内容は 1000 トークンとは限りません。


In [ ]:
response = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1000,
    messages=[{"role": "user", "content": "Tell me a joke"}]
)


In [ ]:
print(response.content[0].text)


In [ ]:
print(response.usage.output_tokens)


上の例では、Claude に "Tell me a joke" と頼み、`max_tokens` に 1000 を指定しました。Claude は次のジョークを生成しました：

```
Here's a classic dad joke for you:

Why don't scientists trust atoms? Because they make up everything!

How was that? I tried to keep it clean and mildly amusing. Let me know if you'd like to hear another joke.
```

この生成内容は 55 トークンでした。1000 トークンという上限を与えたからといって、必ず 1000 トークン生成されるわけではありません。


### なぜ max_tokens を調整するのか？

Claude を扱う上でトークンの理解が重要なのは、主に次の理由からです：

* **API 制限**: 入力テキストと生成された応答のトークン数は API 利用制限に影響します。各リクエストには処理できるトークン数の上限があり、トークンを意識すると制限内に収めやすくなります。
* **性能（パフォーマンス）**: Claude が生成するトークン数は処理時間やメモリ使用量に直接影響します。入力が長い・`max_tokens` が大きいほど計算資源が必要になります。
* **応答品質**: 適切な `max_tokens` を設定すると、必要な情報を含む十分な長さの応答を得やすくなります。小さすぎると途中で切れて不完全になり得ます。ユースケースに合わせて `max_tokens` を調整することで最適なバランスを探せます。


Claude が生成するトークン数がパフォーマンスに与える影響を見てみましょう。次の関数は、2人の登場人物の非常に長い対話を生成させるリクエストを3回行います（それぞれ `max_tokens` の値を変えます）。そして、実際に生成されたトークン数と生成にかかった時間を表示します：


In [ ]:
import time

def compare_num_tokens_speed():
    token_counts = [100, 1000, 4096]
    task = """
        Create a long, detailed dialogue that is at least 5000 words long between two characters discussing the impact of social media on mental health.
        The characters should have differing opinions and engage in a respectful thorough debate.
    """

    for num_tokens in token_counts:
        start_time = time.time()

        response = client.messages.create(
            model="claude-3-haiku-20240307",
            max_tokens=num_tokens,
            messages=[{"role": "user", "content": task}]
        )

        end_time = time.time()
        execution_time = end_time - start_time

        print(f"Number of tokens generated: {response.usage.output_tokens}")
        print(f"Execution Time: {execution_time:.2f} seconds\n")


In [ ]:
compare_num_tokens_speed()


上のコードを実行すると、具体的な数値は毎回変わりますが、出力例は例えば次の通りです：

```
Number of tokens generated: 100
Execution Time: 1.51 seconds

Number of tokens generated: 1000
Execution Time: 8.33 seconds

Number of tokens generated: 3433
Execution Time: 28.80 seconds
```

見ての通り、**Claude が生成するトークン数が増えるほど時間がかかります。**


さらに分かりやすい例として、非常に長いテキストを Claude にそのまま繰り返させ、`max_tokens` で出力サイズをいくつかの段階で打ち切りました。各サイズについて 50 回ずつ繰り返して平均の生成時間を計算しています。出力サイズが大きくなるほど時間が増えることが分かります：

![output_length](images/output_length.png)


## Stop sequences

もう 1 つ、まだ扱っていない重要なパラメータが `stop_sequence`（実際の指定は `stop_sequences`）です。これは、モデルが生成中に特定の文字列に遭遇したら生成を止めるための仕組みです。つまり Claude に対して「このシーケンスを生成したら、それ以上は何も生成せず止めて」と伝える方法です。

まずは `stop_sequence` を指定しない例を見てみましょう：


In [ ]:
response = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=500,
    messages=[{"role": "user", "content": "Generate a JSON object representing a person with a name, email, and phone number ."}],
)
print(response.content[0].text)


上のコードは、名前・メール・電話番号を持つ人物を表す JSON オブジェクトを生成させています。Claude の出力例は次のようになります：

```
Here's an example of a JSON object representing a person with a name, email, and phone number:

{
  "name": "John Doe",
  "email": "johndoe@example.com",
  "phoneNumber": "123-456-7890"
}


In this example, the JSON object has three key-value pairs:

1. "name": The person's name, which is a string value of "John Doe".
2. "email": The person's email address, which is a string value of "johndoe@example.com".
3. "phoneNumber": The person's phone number, which is a string value of "123-456-7890".

You can modify the values to represent a different person with their own name, email, and phone number.
```

欲しい JSON は生成できていますが、その後に説明文も付いています。もし `}` を生成した時点で止めたいなら、`stop_sequences` を追加できます。


In [ ]:
response = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=500,
    messages=[{"role": "user", "content": "Generate a JSON object representing a person with a name, email, and phone number ."}],
    stop_sequences=["}"]
)
print(response.content[0].text)


モデルは次のような出力を生成します：

```
Here's a JSON object representing a person with a name, email, and phone number:

{
  "name": "John Doe",
  "email": "john.doe@example.com",
  "phone": "555-1234"
```

**重要:** この出力には、停止条件として指定した `}` 自体は **含まれません**。JSON としてパースして使いたい場合は、最後に `}` を自分で補う必要があります。


Claude からレスポンスが返ってきたら、`stop_reason` を確認することで、なぜ生成が停止したのかが分かります。先ほどのレスポンスは `stop_reason` が `stop_sequence` になっているはずで、これは指定した停止シーケンスのどれかを生成したため、直ちに停止したことを意味します。


In [ ]:
response.stop_reason


また、レスポンスの `stop_sequence` を見ると、実際にどの停止シーケンスが生成されて停止したのかを確認できます：


In [ ]:
response.stop_sequence


停止シーケンスは複数指定できます。複数指定した場合、モデルはそのうち **どれか 1 つ**に遭遇した時点で停止します。また、レスポンスの `stop_sequence` には、遭遇した停止シーケンスが入ります。

次の関数は、Claude に詩を書かせ、文字 "b" または "c" を生成したら止める、というリクエストを 3 回行います：


In [ ]:
def generate_random_letters_3_times():
    for i in range(3):
        response = client.messages.create(
            model="claude-3-haiku-20240307",
            max_tokens=500,
            messages=[{"role": "user", "content": "generate a poem"}],
            stop_sequences=["b", "c"]
        )
        print(f"Response {i+1} stopped because {response.stop_reason}.  The stop sequence was {response.stop_sequence}")


In [ ]:
generate_random_letters_3_times()


出力例：

```
Response 1 stopped because stop_sequence.  The stop sequence was c
Response 2 stopped because stop_sequence.  The stop sequence was b
Response 3 stopped because stop_sequence.  The stop sequence was b
```

最初は文字 "c" を生成したため停止し、次の 2 回は "b" を生成したため停止しています。こんな指定を実務で使うことはたぶんありませんが、停止シーケンスの仕組みを理解するためのデモです。


## Temperature

`temperature` パラメータは、生成される応答の「ランダムさ」や「創造性」をコントロールするために使います。値は 0〜1 の範囲で、高いほど多様で予測しにくい（言い回しの揺れが増える）応答になりやすく、低いほど最も起こりやすい言い回しや答えに寄った、より決定的（deterministic）な応答になりやすいです。**temperature のデフォルト値は 1** です。

テキスト生成では、Claude は次のトークン（単語やサブワード）の確率分布を予測します。temperature は、次のトークンをサンプリングする前に、その確率分布を調整するために使われます。

- temperature が低い（0.0 に近い）と、確率分布が鋭くなり、最も起こりやすいトークンに高い確率が集中します。その結果、より決定的で「安全側」の選択をしやすくなります。
- temperature が高い（1.0 に近い）と、確率分布が平坦になり、起こりにくいトークンの確率も相対的に上がります。その結果、よりランダムで探索的になり、多様で創造的な出力が出やすくなります。

temperature の影響を視覚化した図はこちらです：

![temperature](images/temperature.png)

なぜ temperature を変えるのか？

**分析的なタスクでは 0.0 に近く、創造的・生成的なタスクでは 1.0 に近い値を使うのがおすすめです。**


簡単なデモをしてみましょう。次の関数では temperature を 0 と 1 にして、それぞれ 3 回ずつ Claude にリクエストを送り、"Come up with a name for an alien planet. Respond with a single word."（エイリアンの惑星名を 1 単語で考えて）と頼みます。


In [ ]:
def demonstrate_temperature():
    temperatures = [0, 1]
    for temperature in temperatures:
        print(f"Prompting Claude three times with temperature of {temperature}")
        print("================")
        for i in range(3):
            response = client.messages.create(
                model="claude-3-haiku-20240307",
                max_tokens=100,
                messages=[{"role": "user", "content": "Come up with a name for an alien planet. Respond with a single word."}],
                temperature=temperature
            )
            print(f"Response {i+1}: {response.content[0].text}")
        


In [ ]:
demonstrate_temperature()


上の関数を実行した結果（あなたの実行結果は多少変わるかもしれません）：

```
Prompting Claude three times with temperature of 0
================
Response 1: Xendor.
Response 2: Xendor.
Response 3: Xendor.
Prompting Claude three times with temperature of 1
================
Response 1: Xyron.
Response 2: Xandar.
Response 3: Zyrcon.
```

temperature が 0 のときは 3 回とも同じ応答になっています。なお temperature が 0.0 でも完全に決定的になるとは限りませんが、temperature が 1 の場合と比べると違いは明確で、毎回まったく別の惑星名が返っています。


temperature が Claude の出力に与える影響を示す別の図です。プロンプト "Pick any animal in the world. Respond with only a single word: the name of the animal"（世界中の動物を 1 単語で答えて）を使い、temperature=0 で 100 回、temperature=1 でさらに 100 回問い合わせました。下のプロットは、Claude が返した動物名の頻度を表します。

![temperature_plot](images/temperature_plot.png)

temperature=0 では毎回 "Giraffe" になっています。temperature=0 は決定性を保証するものではありませんが、毎回似た内容を返しやすくします。temperature=1 では "Giraffe" が半分以上を占めつつも、他の動物も多く含まれています。


## System prompt

`system_prompt` は、Claude にメッセージを送るときに指定できる **任意のパラメータ**です。会話の前提（舞台）を設定するために、高レベルの指示、役割定義、背景情報などを与え、応答に反映させるのに使います。

system prompt のポイント：

* 任意ですが、会話のトーンや文脈を設定するのに便利です。
* 会話（そのやり取り）全体に適用され、その交換内の Claude の応答すべてに影響します。
* 毎回の user メッセージに細かい指示を書かなくても、振る舞いを誘導できます。

基本的には、system prompt にはトーン・文脈・役割に関する内容を入れるのがよいです。詳細な手順や外部入力（ドキュメントなど）、例は最初の `user` ターンに入れるほうが結果が良くなりがちです。以降の `user` ターンで毎回繰り返す必要はありません。

試してみましょう：


In [ ]:
message = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1000,
    system="You are a helpful foreign language tutor that always responds in French.",
    messages=[
        {"role": "user", "content": "Hey there, how are you?!"}
    ]
)

print(message.content[0].text)


***

## 演習

`generate_questions` という関数を書いてください。要件は次の通りです：

* 2 つのパラメータ `topic` と `num_questions` を受け取る
* 与えられた `topic` について、`num_questions` 個の「考えさせられる質問」を **番号付きリスト**として生成する
* 生成した質問を print する

たとえば `generate_questions(topic="free will", num_questions=3)` を呼び出すと、次のような出力になり得ます：

> 1. To what extent do our decisions and actions truly originate from our own free will, rather than being shaped by factors beyond our control, such as our genes, upbringing, and societal influences?
> 2. If our decisions are ultimately the result of a complex interplay of biological, psychological, and environmental factors, does that mean we lack the ability to make authentic, autonomous choices, or is free will compatible with determinism?
> 3. What are the ethical and philosophical implications of embracing or rejecting the concept of free will? How might our views on free will impact our notions of moral responsibility, punishment, and the nature of the human condition?

実装では、次のパラメータを活用してください：

* `max_tokens`: 応答を 1000 トークン未満に制限する
* `system`: system prompt を与えて、モデルが `topic` の専門家として番号付きリストを生成するようにする
* `stop_sequences`: 質問の数が正しいところで止まるようにする（3問なら "4." を生成した時点で止める。5問なら "6." を生成した時点で止める）


#### 解答例


In [ ]:
def generate_questions(topic, num_questions=3):
    response = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=500,
        system=f"You are an expert on {topic}. Generate thought-provoking questions about this topic.",
        messages=[
            {"role": "user", "content": f"Generate {num_questions} questions about {topic} as a numbered list."}
        ],
        stop_sequences=[f"{num_questions+1}."]
    )
    print(response.content[0].text)


In [ ]:
generate_questions(topic="free will", num_questions=3)


***
